In [5]:
import requests
import json
import speech_recognition as sr
import os
from pydub import AudioSegment

def converto_to_wav(input_path, output_path = None):
    """
    오디오 파일을 WAV로 변환하는 함수
    
    :param input_path: 변환할 원본 오디오 파일 경로
    :param output_path: 변환된 WAV 파일 경로 (기본적으로 동일한 폴더에 저장)
    :return: 변환된 WAV 파일 경로
    """
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"파일을 찾을 수 없습니다. {input_path}")
    
    # 출력 경로 설정    
    if output_path is None:
        output_path = os.path.splitext(input_path)[0] + ".wav"
    
    #오디오 파일 로드 및 WAV로 변환
    audio = AudioSegment.from_file(input_path)
    audio.export(output_path, format = "wav")

    print(f"변환 완료: {output_path}")
    return output_path



# ETRI API 키 입력
API_KEY = "41a2a58b-2dae-4d98-b3cc-70a7f0d57887"

# ETRI API 엔드포인트
URL = "http://aiopen.etri.re.kr:8000/WiseNLU"

# 음성을 텍스트로 변환하는 함수
def speech_to_text(audio_file):
    recognizer = sr.Recognizer()
    
    if not os.path.exists(audio_file):
        return "파일이 존재하지 않습니다."

    with sr.AudioFile(audio_file) as source:
        audio_data = recognizer.record(source)

    try:
        text = recognizer.recognize_google(audio_data, language="ko-KR")
        return text
    except sr.UnknownValueError:
        return "음성을 인식할 수 없습니다."
    except sr.RequestError as e:
        return f"Google API 요청 오류: {e}"

# 텍스트 감정 분석 요청 함수    
def analyze_emotion(text):
    request_body = {
        "access_key": API_KEY,
        "argument": {
            "text": text,
            "analysis_code": "SA"
        }
    }
    response = requests.post(URL, json=request_body)
    
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": f"API 요청 실패 (HTTP {response.status_code})"}

# 음성 파일 입력
audio_file = "/home/usou/dev_ws/superbad/data/usou/record/test_2.m4a"  # 분석할 음성 파일

# 음성을 텍스트로 변환
recognized_text = speech_to_text(converto_to_wav(audio_file))
print(f"인식된 텍스트: {recognized_text}")

# 오류 메시지가 아닌 경우 감정 분석 실행
if recognized_text and "오류" not in recognized_text and "없습니다" not in recognized_text:
    sentiment_result = analyze_emotion(recognized_text)
    print(json.dumps(sentiment_result, indent=4, ensure_ascii=False))
else:
    print("텍스트 변환 실패로 인해 감정 분석을 수행하지 않습니다.")


변환 완료: /home/usou/dev_ws/superbad/data/usou/record/test_2.wav
인식된 텍스트: 50분 알람
{
    "error": "API 요청 실패 (HTTP 403)"
}


# 형태소 분석 테스트 코드

In [5]:
import requests
import json

# ETRI API 키
API_KEY = "41a2a58b-2dae-4d98-b3cc-70a7f0d57887"  # 본인의 키 입력

# ETRI 형태소 분석 API 엔드포인트
URL = "http://aiopen.etri.re.kr:8000/WiseNLU"

# 요청 데이터
request_body = {
    "access_key": API_KEY,
    "argument": {
        "text": "오늘 날씨가 좋네요.",  # 테스트 문장
        "analysis_code": "morp"  # 형태소 분석 코드
    }
}

# API 요청 실행
response = requests.post(URL, json=request_body)

# 결과 출력
print("형태소 분석 API 응답 코드:", response.status_code)
print("형태소 분석 API 응답 결과:", response.json())


형태소 분석 API 응답 코드: 403
형태소 분석 API 응답 결과: {'result': -1, 'reason': 'Empty Auth Header'}
